In [1]:
import pypyodbc
import pandas

# Define the Components of the Connection String.
server = '192.168.0.21'
port = '4900'
database = 'BAC_A_SABLE'
username = 'admin'
password = 'Clip_SERENA'

# Create a connection object.
connection_object: pypyodbc.Connection = pypyodbc.connect('DRIVER={HFSQL}; \
                        Server Name =' + server + '; \
                        Server Port=' + port + '; \
                        DATABASE=' + database + '; \
                        UID=' + username + '; \
                        PWD=' + password)
                        #Trusted_Connection=yes;

# Create a Cursor Object, using the connection.
cursor_object: pypyodbc.Cursor = connection_object.cursor()

table_name = 'CHARGES'
table_name_2 = 'CFRAIS'
site = 'ATCRM'
atelier = 'TOUR'
annee = '2024'
semaine = '12'

# Define the Select Query.
sql_select = "SELECT "+table_name+".COSECT, "+table_name+".ANNEE, "+table_name+".SEMAINE, "+table_name+".COFRAIS, "+table_name_2+".DESIGN, "+table_name+".VDUREE  \
                FROM "+table_name+" LEFT JOIN "+table_name_2+" ON "+table_name+".COFRAIS = "+table_name_2+".COFRAIS \
                WHERE COSECT like '"+site+"' AND ANNEE = "+annee+" AND SEMAINE = "+semaine+" AND "+table_name_2+".DESIGN like '%"+atelier+"%'"
cursor_object.execute(sql_select)

# Define the column names.
#columns = [column[0] for column in cursor_object.statistics]
columns = [column[0] for column in cursor_object.description]

# Execute the Query.
records = cursor_object.fetchall()

# Dump to a Pandas DataFrame.
plancharge_df = pandas.DataFrame.from_records(
    data=records,
    columns=columns
)

charge_list = plancharge_df.to_dict('records')
# print the head.
print(charge_list)

[{'cosect': 'ATCRM', 'annee': 2024, 'semaine': 12, 'cofrais': 'CP700', 'design': 'CRM TOUR CN PUMA 700          ', 'vduree': 68.961965}, {'cosect': 'ATCRM', 'annee': 2024, 'semaine': 12, 'cofrais': 'CP480', 'design': 'CRM TOUR CN PUMA 480          ', 'vduree': 275.348895}, {'cosect': 'ATCRM', 'annee': 2024, 'semaine': 12, 'cofrais': 'CTOUR', 'design': 'CRM TOURNAGE CONVENTIONNEL    ', 'vduree': 4.050289}, {'cosect': 'ATCRM', 'annee': 2024, 'semaine': 12, 'cofrais': 'C35-2', 'design': 'CRM TOUR CMZ N°2              ', 'vduree': 149.558448}, {'cosect': 'ATCRM', 'annee': 2024, 'semaine': 12, 'cofrais': 'CQTS ', 'design': 'CRM TOUR CN MAZAK QTS 200     ', 'vduree': 141.803715}]


In [2]:
import json

# Convertir le DataFrame en un dictionnaire de groupes de données.
grouped_data = donnees_affaires.groupby(["cosect", "annee", "semaine"])

# Créer une liste de dictionnaires JSON avec la structure requise.
json_data = []
for (cosect, annee, semaine), group in grouped_data:
    postes = []
    for index, row in group.iterrows():
        poste = {
            "cofrais": row["cofrais"].strip(),  # Supprimer les espaces avant et après le code de frais
            "design": row["design"],
            "vduree": row["vduree"]
        }
        postes.append(poste)
    data_entry = {
        "cosect": cosect,
        "annee": int(annee),  # Convertir en entier
        "semaine": int(semaine),  # Convertir en entier
        "postes": postes
    }
    json_data.append(data_entry)

# Charger la chaîne JSON en tant qu'objet JSON
#data = json.loads(json_data)

print(json_data)

[{'cosect': 'ATCRM', 'annee': 2024, 'semaine': 12, 'postes': [{'cofrais': 'CP700', 'design': 'CRM TOUR CN PUMA 700          ', 'vduree': 68.961965}, {'cofrais': 'CP480', 'design': 'CRM TOUR CN PUMA 480          ', 'vduree': 275.348895}, {'cofrais': 'CTOUR', 'design': 'CRM TOURNAGE CONVENTIONNEL    ', 'vduree': 4.050289}, {'cofrais': 'C35-2', 'design': 'CRM TOUR CMZ N°2              ', 'vduree': 149.558448}, {'cofrais': 'CQTS', 'design': 'CRM TOUR CN MAZAK QTS 200     ', 'vduree': 141.803715}]}]
